# NYT Spelling Bee Solver

This notebook solves the New York Times Spelling Bee game using the local `nytbee_dict.txt` word list.

**Rules summary (from the NYT Spelling Bee instructions):**
- Use the seven given letters to make words.
- Words must be at least 4 letters long.
- The first (center) letter is required in every word.
- Letters can be reused.

Enter the letters when prompted with the required letter first. The solver prints answers grouped by word length, similar to the NYT Spelling Bee hint page.


In [ ]:
from __future__ import annotations

from collections import defaultdict
from pathlib import Path


def load_words(path: Path) -> list[str]:
    if not path.exists():
        raise FileNotFoundError(f"Missing wordlist: {path}")
    contents = path.read_text().strip()
    if not contents:
        return []
    if contents.lstrip().startswith("{"):
        from ast import literal_eval

        try:
            loaded = literal_eval(contents)
        except (ValueError, SyntaxError):
            loaded = None
        if isinstance(loaded, dict):
            words = [str(key).strip().lower() for key in loaded]
            return [word for word in words if word.isalpha()]

    words = [line.strip().lower() for line in contents.splitlines()]
    return [word for word in words if word.isalpha()]


def normalize_letters(raw_letters: str) -> tuple[str, str]:
    cleaned = [char for char in raw_letters.lower() if char.isalpha()]
    if not cleaned:
        raise ValueError("Please provide the seven Spelling Bee letters.")

    required = cleaned[0]
    unique_letters = []
    seen = set()
    for char in cleaned:
        if char not in seen:
            seen.add(char)
            unique_letters.append(char)

    if len(unique_letters) < 7:
        raise ValueError("Please provide seven unique letters (required letter first).")

    return required, ''.join(unique_letters)


def solve_spelling_bee(letters: str, wordlist_path: Path) -> tuple[list[str], list[str], str, str]:
    required, cleaned_letters = normalize_letters(letters)
    allowed = set(cleaned_letters)

    words = [
        word
        for word in load_words(wordlist_path)
        if len(word) >= 4 and required in word and set(word) <= allowed
    ]
    pangrams = [word for word in words if set(cleaned_letters) <= set(word)]
    return sorted(words), sorted(pangrams), cleaned_letters, required


def print_hint_page(words: list[str], pangrams: list[str], letters: str, required: str) -> None:
    print('NYT Spelling Bee Hint Page')
    print('-' * 30)
    print(f"Letters: {', '.join(letters)} (required: {required})")
    print(f"Total words: {len(words)}")
    print(f"Pangrams ({len(pangrams)}): {', '.join(pangrams) if pangrams else 'None'}")

    grouped: dict[int, list[str]] = defaultdict(list)
    for word in words:
        grouped[len(word)].append(word)

    print('\nBy length:')
    for length in sorted(grouped):
        word_list = ', '.join(sorted(grouped[length]))
        print(f"{length} letters ({len(grouped[length])}): {word_list}")

    print('\nAlphabetical:')
    print(', '.join(sorted(words)))


In [ ]:
letters_input = input('Enter the Spelling Bee letters (required letter first): ')
words, pangrams, cleaned_letters, required_letter = solve_spelling_bee(letters_input, Path('nytbee_dict.txt'))
print_hint_page(words, pangrams, cleaned_letters, required_letter)
